`TURTLEBOT3_MODEL=waffle_rgp roslaunch turtlebot3_gazebo turtlebot3_rgp.launch`

In [1]:
import rospy as rp
import jupyros as jr
import numpy as np

In [2]:
from geometry_msgs.msg import Pose
rp.init_node('runner')

In [3]:
from geometry_msgs.msg import Pose, Twist
import math
from sensor_msgs.msg import Image,Illuminance,LaserScan
import cv2

In [4]:
def getIlluminance(imgdata,width,height):
    _fov = 6
    startingPix = width * (height//2 -  _fov//2) - _fov//2;

    illum = 0;
    for i in range(_fov):
        index = startingPix + i*width
        for j in range(_fov):
            illum += imgdata[index+j];

    return illum/(_fov*_fov)

In [5]:
speed = 0
turn = 0

In [6]:
print(speed,turn)

0 0


In [7]:
class PID:
    def __init__(self,Kp,Ki,Kd):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.lastError = 0
        self.I = 0
        
    def value(self,e):
        P = e
        self.I = self.I + e
        D = e - self.lastError
        
        self.lastError = e
        
        return P*self.Kp+self.I*self.Ki+D*self.Kd

In [8]:
lfPID = PID(0.3,0,-0.08)

def drive(ls):    
    global speed
    global turn
    
    if min(ls) > 80:
        speed = -0.1
        turn = 0
    else:
        k = [l if l < 55 else 155 for l in ls]
        error = (8-np.argmin(ls))#8 - sum([i*ls[i] for i in range(len(ls))])/sum(ls)
        turn =lfPID.value(error)
        
        #if turn > 1:
        #    turn = 1
        #elif turn < -1:
        #    turn = -1
        
        speed = max(0.3-abs(turn)*0.15,0)
        
    return speed,turn

In [9]:
control_turn

NameError: name 'control_turn' is not defined

In [10]:
qs

NameError: name 'qs' is not defined

In [11]:
np.argmin(qs)

NameError: name 'qs' is not defined

In [12]:
d = True

In [13]:
avoidance=0

In [14]:
oPID = PID(1e-2,1e-5,1e-4)

In [15]:
def front_lidar(ranges,ls,a):
    global d
    global sa 
    
    val = np.min(ranges[:30]+ranges[-30:])
    
    if val < 0.35:
        if left < right:
            if a == 1:
                d=False
            return a+1,0.0,-0.5
        else:
            if a == 1:
                d=True
            return a+1,0.0,0.5

In [35]:
oPID = PID(2e-3,1e-8,1e-2)#1e-5,1e-4)

In [42]:

def handle_scan(ranges,ls,a):
    global d
    
    left = ranges[60]
    right = ranges[-60]
    
    val = np.min(ranges[:30]+ranges[-30:])

    if a != 1 and val < 0.35:
        if left < right:
            if a == 0:
                d=False
            return 1,0.0,0.0
        else:
            if a == 0:
                d=True
            return 1,0.0,0.0
    elif a == 1:
        val = np.min(ranges[:30]+ranges[-30:])
    
        if val < 0.35:
            return a,-0.1,0.0
        else:
            return a+1,0,0
    elif a == 2:
        mi = np.argmin(ranges) 
        
        if mi > 80 and mi < 100 or mi > 260 and mi < 280:
            return a+1,0.2,0
        elif not d:
            return a,0.0,-0.5
        else:
            return a,0.0,0.5
    elif a == 3:
        if np.min(ls) < 50:
            return a,0.2,0
        else:
            return a+1,0,0
    elif a == 4:      
        #return a,0,0
        if np.min(ls) < 30:
            return a+1,0,0 
        
        idx = 90 if not d else 270
        
        #vals = np.array([v if v != np.inf else 10 for v in ranges])
        
        #min_dist = min(np.min(vals),0.8)
        #amin = np.argmin(vals)
                          
        #if min_dist < 0.2:
        #    return a,0.05,0.1 if d else -0.1
        #elif min_dist > 0.35:
        #    error =  (idx - np.argmin(vals))#0.5-np.mean(vals[idx-30:idx+30]*1000)#(idx - np.argmin(vals))#0.5-np.mean(vals[idx-20:idx+20])#(idx - np.argmin(vals))
        #
        #    turn = -oPID.value(error)
        #    return a,0.3,turn
        
        r = np.array([v if v < 1 else 10 for v in sc[idx:]+sc[:idx]])
        
        error =  (180-np.argmin(r-0.15))+(np.min(r)-0.15)*2000*(-1 if d else 1)#(idx - np.argmin(vals))#0.5-np.mean(vals[idx-30:idx+30]*1000)#(idx - np.argmin(vals))#0.5-np.mean(vals[idx-20:idx+20])#(idx - np.argmin(vals))
        
        turn = oPID.value(error)
        
        if turn > 1:
            turn = 1
        elif turn < -1:
            turn = -1
        
        speed = max(0.3-abs(turn)*1.2,0.1)
        
        return a,speed,turn
        #if not d:
        #    m = np.min(ranges[80:100])
        #    return a,0.2,m/4 if m > 0.5 and np.max(ranges[80:100]) < 2 else -0.1
        #else:
        #    m = np.min(ranges[260:280])
        #    return a,0.2,-m/4 if m > 0.5 and np.max(ranges[260:280]) < 2 else 0.1
    elif a == 5:  
        mi = np.argmin(ranges) 
        
        if mi > 160 and mi < 200 or min(ls) > 50:
            return 0,0,0
        elif not d:
            return a,0.05,-0.5
        else:
            return a,0.05,0.5
        
    return 0,0,0 

In [18]:
qs

NameError: name 'qs' is not defined

In [19]:
avoidance=0

In [20]:
control_speed

NameError: name 'control_speed' is not defined

In [21]:
from multiprocessing import Queue
from functools import partial 

stop = False

speed = 0.1
turn = 0.5

control_speed=0
control_turn=0

thresh_illum = 100

n = 16

pub = rp.Publisher('/cmd_vel', Twist,queue_size=10)

qs = [0]*n

def setQi(x,i):
    global qs
    qs[i] = x.illuminance
    
sc = None

def setSc(x):
    global sc
    sc = x.ranges
    
cams = [rp.Subscriber("/camera_{}/rgb/image_raw".format(i),Image) for i in range(n)]
subs = [rp.Subscriber("/light_sensor_plugin/lightSensor/camera_{}".format(i),Illuminance,partial(lambda x,k:setQi(x,k),k=i)) for i in range(n)]
scans = rp.Subscriber("/scan",LaserScan,setSc)

In [22]:
%%thread_cell

import time

last_speed = 0
last_turn = 0

print("start")

avoidance = 0

while not stop:
    #if qright is None or qleft is None:
    #    time.sleep(0.1)
    #    continue
        
    #dIllum = illumLeft - illumRight
    
    #if dIllum > thresh_illum:
    #    control_speed = 0
    #    control_turn = -turn
    #elif dIllum < -thresh_illum:
    #    control_speed = 0
    #    control_turn = turn
    #else:
    #    control_speed = min(0.15,control_speed+0.005)#min(speed,control_speed+0.01) 
    #    control_turn *= 0.8
    
    #if last_speed != control_speed or last_turn != control_turn:
    avoidance,control_speed,control_turn = handle_scan(sc,qs,avoidance)
    
    if avoidance == 0:
        (control_speed,control_turn) = drive(qs)
    
    twist = Twist()
    twist.linear.x = control_speed; twist.linear.y = 0; twist.linear.z = 0
    twist.angular.x = 0; twist.angular.y = 0; twist.angular.z = control_turn
    pub.publish(twist)
            
    last_speed = control_speed
    last_turn = control_turn
    
    time.sleep(0.01)
    
twist = Twist()
twist.linear.x = 0; twist.linear.y = 0; twist.linear.z = 0
twist.angular.x = 0; twist.angular.y = 0; twist.angular.z = 0
pub.publish(twist)
    
print("unregistering")
for cam in cams:
    cam.unregister()
    
for sub in subs:
    sub.unregister()
pub.unregister()
scans.unregister()
print("done")

Output(layout=Layout(border='1px solid gray'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': 's…

In [43]:
stop = True

In [221]:
for cam in cams:
    cam.unregister()
    

for sub in subs:
    sub.unregister()
pub.unregister()
scans.unregister()

In [207]:
0.1*1.1**10

0.25937424601000025

In [24]:
qs

[201.0,
 201.0,
 201.0,
 201.0,
 201.0,
 168.16666666666666,
 102.83333333333333,
 102.83333333333333,
 70.33333333333333,
 70.33333333333333,
 102.83333333333333,
 102.83333333333333,
 168.16666666666666,
 201.0,
 201.0,
 201.0]

In [28]:
len(qs)

16

In [30]:
control_turn

3.149033612103871e-05

In [288]:
avoidance


1

In [86]:
control_speed

0.1

nan

In [55]:
sc

(inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 inf,
 0.9049496054649353,
 0.8850511312484741,
 0.8630180954933167,
 0.8828028440475464,
 0.8545817732810974,
 0.8409129977226257,
 0.846435010433197,
 0.8359984159469604,
 0.8141589164733887,
 0.8099699020385742,
 0.7973828315734863,
 0.794128954410553,
 0.788258492946624

In [120]:
sc[0]==np.inf

True